This lab on Support Vector Machines in R is an abbreviated version of p. 359-366 of "Introduction to Statistical
Learning with Applications in R" by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani.

# 9.6 Lab: Support Vector Machines

In this lab, we'll use the ${\tt e1071}$ library in ${\tt R}$ to demonstrate the support vector classifier
and the SVM. Another option is the ${\tt LiblineaR}$ library, which is particularly useful for
very large linear problems.

# 9.6.1 Support Vector Classifier
The ${\tt e1071}$ library contains implementations for a number of statistical
learning methods. In particular, the ${\tt svm()}$ function can be used to fit a
support vector classifier when the argument ${\tt kernel="linear"}$ is used. This
function uses a slightly different formulation of the equations we saw in lecture to build the
support vector classifier. A ${\tt cost}$ argument allows us to specify the cost of
a violation to the margin. When the ${\tt cost}$ argument is **small**, then the margins
will be wide and many support vectors will be on the margin or will
violate the margin. When the ${\tt cost}$ argument is large, then the margins will
be narrow and there will be few support vectors on the margin or violating
the margin.

We can use the ${\tt svm()}$ function to fit the support vector classifier for a
given value of the ${\tt cost}$ parameter. Here we demonstrate the use of this
function on a two-dimensional example so that we can plot the resulting
decision boundary. Let's start by generating a set of observations, which belong
to two classes:

In [ ]:
set.seed(1)
x=matrix(rnorm(20*2), ncol=2)
y=c(rep(-1,10), rep(1,10))
x[y==1,]=x[y==1,] + 1

Let's plot the data to see whether the classes are linearly separable:

In [ ]:
plot(x, col=(3-y))

Nope; not linear. Next, we fit the support vector classifier. Note that in order
for the ${\tt svm()}$ function to perform **classification** (as opposed to SVM-based
regression), we must encode the response as a **factor**:

In [ ]:
dat=data.frame(x=x, y=as.factor(y))
library(e1071)
svmfit=svm(y~., data=dat, kernel="linear", cost=10, scale=FALSE)

The argument ${\tt scale=FALSE}$ tells the ${\tt svm()}$ function not to scale each feature
to have mean zero or standard deviation one; depending on the application,
we might prefer to use ${\tt scale=TRUE}$. We can now plot the support vector classifier by calling the ${\tt plot()}$ function on the output of the call to ${\tt svm()}$, as well as the data used in the call to ${\tt svm()}$:

In [ ]:
plot(svmfit, dat)

The region of feature space that will be assigned to the −1 class is shown in
light blue, and the region that will be assigned to the +1 class is shown in
purple. The decision boundary between the two classes is linear (because we
used the argument ${\tt kernel="linear"}$), though due to the way in which the
plotting function is implemented in this library the decision boundary looks
somewhat jagged in the plot. We see that in this case only one observation
is misclassified. (Note also that the second feature is plotted on the x-axis
and the first feature is plotted on the y-axis, in contrast to the behavior of
the usual ${\tt plot()}$ function in ${\tt R}$.) 

The support vectors are plotted as crosses
and the remaining observations are plotted as circles; we see here that there
are seven support vectors. We can determine their identities as follows:

In [ ]:
svmfit$index

We can obtain some basic information about the support vector classifier
fit using the ${\tt summary()}$ command:

In [ ]:
summary(svmfit)

This tells us, for instance, that a linear kernel was used with ${\tt cost=10}$, and
that there were seven support vectors, four in one class and three in the
other. What if we instead used a smaller value of the ${\tt cost}$ parameter?

In [ ]:
svmfit=svm(y~., data=dat, kernel="linear", cost=0.1, scale=FALSE)
plot(svmfit, dat)
svmfit$index

Now that a smaller value of the ${\tt cost}$ parameter is being used, we obtain a
larger number of support vectors, because the margin is now **wider**. Unfortunately,
the ${\tt svm()}$ function does not explicitly output the coefficients of
the linear decision boundary obtained when the support vector classifier is
fit, nor does it output the width of the margin.

The ${\tt e1071}$ library includes a built-in function, ${\tt tune()}$, to perform cross-validation. By default, ${\tt tune()}$ performs **ten-fold cross-validation** on a set
of models of interest. In order to use this function, we pass in relevant
information about the set of models that are under consideration. The
following command indicates that we want to compare SVMs with a linear
kernel, using a range of values of the cost parameter:

In [ ]:
set.seed(1)
tune.out=tune(svm,y~.,data=dat,kernel="linear",ranges=list(cost=c(0.001, 0.01, 0.1, 1,5,10,100)))

We can easily access the cross-validation errors for each of these models
using the ${\tt summary()}$ command:

In [ ]:
summary(tune.out)

The ${\tt tune()}$ function stores the best model obtained, which can be accessed as
follows:

In [ ]:
bestmod=tune.out$best.model
summary(bestmod)

As usual, the ${\tt predict()}$ function can be used to predict the class label on a set of
test observations, at any given value of the cost parameter. Let's
generate a test data set:

In [ ]:
xtest=matrix(rnorm(20*2), ncol=2)
ytest=sample(c(-1,1), 20, rep=TRUE)
xtest[ytest==1,]=xtest[ytest==1,] + 1
testdat=data.frame(x=xtest, y=as.factor(ytest))

Now we predict the class labels of these test observations. Here we use the
best model obtained through cross-validation in order to make predictions:

In [ ]:
ypred=predict(bestmod,testdat)
table(predict=ypred, truth=testdat$y)

Thus, with this value of ${\tt cost}$, 19 of the test observations are correctly
classified. What if we had instead used ${\tt cost=0.01}$?

In [ ]:
svmfit=svm(y~., data=dat, kernel="linear", cost=.01,scale=FALSE)
ypred=predict(svmfit,testdat)
table(predict=ypred, truth=testdat$y)

In this case one additional observation is misclassified.

Now consider a situation in which the two classes are linearly separable.
Then we can find a separating hyperplane using the ${\tt svm()}$ function. First we'll give our simulated data a little nudge so that they are linearly separable:

In [ ]:
x[y==1,]=x[y==1,]+0.5
plot(x, col=(y+5)/2, pch=19)

Now the observations are **just barely linearly** separable. We fit the support
vector classifier and plot the resulting hyperplane, using a very large value
of ${\tt cost}$ so that no observations are misclassified.

In [ ]:
dat=data.frame(x=x,y=as.factor(y))
svmfit=svm(y~., data=dat, kernel="linear", cost=1e5)
summary(svmfit)
plot(svmfit, dat)

No training errors were made and only three support vectors were used.
However, we can see from the figure that the margin is very narrow (because
the observations that are **not** support vectors, indicated as circles, are very close to the decision boundary). It seems likely that this model will perform
poorly on test data. Let's try a smaller value of ${\tt cost}$:

In [ ]:
svmfit=svm(y~., data=dat, kernel="linear", cost=1)
summary(svmfit)
plot(svmfit,dat)

Using ${\tt cost=1}$, we misclassify a training observation, but we also obtain
a much wider margin and make use of seven support vectors. It seems
likely that this model will perform better on test data than the model with
${\tt cost=1e5}$.

# 9.6.2 Support Vector Machine

In order to fit an SVM using a **non-linear kernel**, we once again use the ${\tt svm()}$
function. However, now we use a different value of the parameter kernel.
To fit an SVM with a polynomial kernel we use ${\tt kernel="polynomial"}$, and
to fit an SVM with a radial kernel we use ${\tt kernel="radial"}$. In the former
case we also use the ${\tt degree}$ argument to specify a degree for the polynomial
kernel, and in the latter case we use ${\tt gamma}$ to specify a
value of $\gamma$ for the radial basis kernel.

Let's generate some data with a non-linear class boundary:

In [ ]:
set.seed(1)
x=matrix(rnorm(200*2), ncol=2)
x[1:100,]=x[1:100,]+2
x[101:150,]=x[101:150,]-2
y=c(rep(1,150),rep(2,50))
dat=data.frame(x=x,y=as.factor(y))
plot(x, col=y)

See how one class is kind of stuck in the middle of another class? This suggests that we might want to use a **radial kernel** in our SVM. Now let's randomly split this data into training and testing groups, and then fit
the training data using the ${\tt svm()}$ function with a radial kernel and $\gamma = 1$:

In [ ]:
train=sample(200,100)
svmfit=svm(y~., data=dat[train,], kernel="radial",  gamma=1, cost=1)
plot(svmfit, dat[train,])

Not too shabby! The plot shows that the resulting SVM has a decidedly non-linear
boundary. We can see from the figure that there are a fair number of training errors
in this SVM fit. If we increase the value of cost, we can reduce the number
of training errors:

In [ ]:
svmfit=svm(y~., data=dat[train,], kernel="radial",gamma=1,cost=1e5)
plot(svmfit,dat[train,])

However, this comes at the price of a more irregular decision boundary that seems to be at risk of overfitting the data. We can perform cross-validation using ${\tt tune()}$ to select the best choice of
$\gamma$ and cost for an SVM with a radial kernel:

In [ ]:
set.seed(1)
tune.out=tune(svm, y~., data=dat[train,], kernel="radial", ranges=list(cost=c(0.1,1,10,100,1000),gamma=c(0.5,1,2,3,4)))
bestmod=tune.out$best.model
summary(bestmod)

Therefore, the best choice of parameters involves ${\tt cost=1}$ and ${\tt gamma=2}$. We
can plot the resulting fit using the ${\tt plot()}$ function, and view the test set predictions for this model by applying the ${\tt predict()}$
function to the data. Notice that to do this we subset the dataframe dat
using ${\tt -train}$ as an index set:

In [ ]:
plot(bestmod,dat[train,])
table(true=dat[-train,"y"], pred=predict(tune.out$best.model,newdata=dat[-train,]))

90% of test observations are correctly classified by this SVM. Not bad!

# 9.6.3 ROC Curves

The ${\tt ROCR}$ package can be used to produce ROC curves such as those we saw in lecture. We first write a short function to plot an ROC curve
given a vector containing a numerical score for each observation, ${\tt pred}$, and
a vector containing the class label for each observation, ${\tt truth}$:

In [ ]:
library(ROCR)
rocplot=function(pred, truth, ...){
   predob = prediction(pred, truth)
   perf = performance(predob, "tpr", "fpr")
   plot(perf,...)}

SVMs and support vector classifiers output class labels for each observation.
However, it is also possible to obtain fitted values for each observation,
which are the numerical scores used to obtain the class labels. For instance,
in the case of a support vector classifier, the fitted value for an observation
$X = (X_1,X_2, . . .,X_p)^T$ takes the form $\hat\beta_0 + \hat\beta_1X_1 + \hat\beta_2X_2 + . . . + \hat\beta_pX_p$.

For an SVM with a non-linear kernel, the equation that yields the fitted
value is given in (9.23) on p. 352 of the ISLR book. In essence, the sign of the fitted value determines
on which side of the decision boundary the observation lies. Therefore, the
relationship between the fitted value and the class prediction for a given
observation is simple: if the fitted value exceeds zero then the observation
is assigned to one class, and if it is less than zero than it is assigned to the
other.

In order to obtain the fitted values for a given SVM model fit, we
use ${\tt decision.values=TRUE}$ when fitting ${\tt svm()}$. Then the ${\tt predict()}$ function
will output the fitted values. Let's fit models using the $\gamma$ selected by cross-validation, and a higher value, which will produce a more flexible fit:

In [ ]:
svmfit.opt=svm(y~., data=dat[train,], kernel="radial",gamma=2, cost=1,decision.values=T)
svmfit.flex=svm(y~., data=dat[train,], kernel="radial",gamma=50, cost=1,decision.values=T)

Now we can produce the ROC plot to see how the models perform on both the training and the test data:

In [ ]:
par(mfrow=c(1,2))

# Plot optimal parameter model's performance on training data
fitted.opt.train=attributes(predict(svmfit.opt,dat[train,],decision.values=TRUE))$decision.values
rocplot(fitted.opt.train,dat[train,"y"],main="Training Data")

# Add more flexible model's performance to the plot
fitted.flex.train=attributes(predict(svmfit.flex,dat[train,],decision.values=TRUE))$decision.values
rocplot(fitted.flex.train,dat[train,"y"], add=T, col="red")

# Plot optimal parameter model's performance on test data
fitted.opt.test=attributes(predict(svmfit.opt,dat[-train,],decision.values=T))$decision.values
rocplot(fitted.opt.test,dat[-train,"y"],main="Test Data")

# Add more flexible model's performance to the plot
fitted.flex.test=attributes(predict(svmfit.flex,dat[-train,],decision.values=TRUE))$decision.values
rocplot(fitted.flex.test,dat[-train,"y"], add=T, col="red")

To get credit for this lab, describe what the ROC plot is telling you about the SVM's performance on the test data and post to Piazza: https://piazza.com/class/igwiv4w3ctb6rg?cid=54